In [1]:
# !pip install plotly panel
# !pip install nbformat 

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [3]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (EUR),Category,Year,Month,Month Name
0,2025-06-26,steam game,Expense,5.0,entertainment,2025,6,Jun
1,2025-06-26,movie,Expense,10.0,entertainment,2025,6,Jun
2,2025-06-26,salary,Income,30.0,personal,2025,6,Jun
3,2024-12-24,christmas gift,Expense,15.0,holiday,2024,12,Dec
4,2025-04-27,birthday gift,Expense,20.0,personal,2025,4,Apr
5,2024-07-26,gift,Expense,10.0,personal,2024,7,Jul
6,2025-06-07,salary,Income,30.0,personal,2025,6,Jun
7,2024-10-31,halloween decoration,Expense,5.0,holiday,2024,10,Oct
8,2025-02-14,valentine's gift,Expense,3.0,personal,2025,2,Feb
9,2025-05-19,gift,Expense,7.0,personal,2025,5,May


In [4]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

### Make pie charts - Income / Expense breakdown

In [5]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2

    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction = 'clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum()
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()

    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10,
                          uniformtext_mode='hide',
                          title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                          # Add annotations in the center of the donut.
                          annotations=[
                              dict(
                                  text=total_text,
                                  # Square unit grid starting at bottom left of page
                                  x=0.5, y=0.5, font_size=12,
                                  # Hide the arrow that points to the [x,y] coordinate
                                  showarrow=False
                              )
                          ]
                          )
    return pie_fig

In [10]:
income_pie_fig_2022 = make_pie_chart(df, 2025, 'Income')
income_pie_fig_2022

### Make bar charts over months in a year

In [7]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                      .to_frame()
                      .reset_index()
                      .sort_values(by='Month')
                      .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    #bar_fig.update_traces(marker_color='lightslategrey')

    return bar_fig

In [11]:
income_monthly_2022 = make_monthly_bar_chart(df, 2025, 'Income')
income_monthly_2022

### Putting all charts together into tabs for 2022/2023

In [12]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2024, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2024, 'Expense')
income_pie_fig_2023 = make_pie_chart(df, 2025, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2025, 'Expense')

# Bar Charts
income_monthly_2022 = make_monthly_bar_chart(df, 2024, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2024, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2025, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2025, 'Expense')

# Create tabs 
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22472\2646171790.py:17: RuntimeWarning:

divide by zero encountered in scalar divide



OverflowError: cannot convert float infinity to integer

### Create Dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"),
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                )
                ],

    # accent_base_color="#88d8b8",
    header_background="c0b9dd",
)

template.show()